In [32]:
from pyspark.sql.types import StructType
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import math

In [33]:
my_conf = SparkConf()
my_conf.set("spark.app.name", "w12_Spark_1")
my_conf.set("spark.master", "local[*]")
my_conf.set("spark.driver.extraClassPath", "C:/Users/Vinoth/Downloads/sqljdbc_12.4.2.0_enu/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre8.jar")

spark = SparkSession.builder.config(conf = my_conf).getOrCreate()
spark


In [34]:
database_url = "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true"

database_name = "Learn_Sql"

jdbc_properties = {

"user": "Vinoth",

"password": "1234",

"driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"

}

In [35]:
# df_min_max = (spark.read
# .format("jdbc")
# .option("url", "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true;databaseName=Learn_Sql;")
# .option("query","select min(cost) as min_row, max(cost) as max_row from products")
# .option("user", "Vinoth")
# .option("password", "1234")
# .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
# .load().collect())

# min_value, max_value = df_min_max[0][0], df_min_max[0][1]

In [36]:
# df = (spark.read
#       .option("numPartitions", 3)
#       .option("PartitionColumn", "cost")
#       .option("lowerBound", int(min_value))
#       .option("upperBound", int(max_value))
#       .jdbc(url=f"{database_url};databaseName={database_name};", table="products", properties=jdbc_properties))

In [37]:
hospital_df = (spark.read
      .jdbc(url=f"{database_url};databaseName={database_name};", table="hospital", properties=jdbc_properties))

In [38]:
hospital_df.show()

+------+------+-------------------+
|emp_id|action|               time|
+------+------+-------------------+
|     1|    in|2019-12-22 09:00:00|
|     1|   out|2019-12-22 09:15:00|
|     2|    in|2019-12-22 09:00:00|
|     2|   out|2019-12-22 09:15:00|
|     2|    in|2019-12-22 09:30:00|
|     3|   out|2019-12-22 09:00:00|
|     3|    in|2019-12-22 09:15:00|
|     3|   out|2019-12-22 09:30:00|
|     3|    in|2019-12-22 09:45:00|
|     4|    in|2019-12-22 09:45:00|
|     5|   out|2019-12-22 09:40:00|
+------+------+-------------------+



In [39]:
in_time = hospital_df.where("action = 'in'").groupBy('emp_id', "action").agg(expr("max(time) as in_max_time"))

In [40]:
out_time = hospital_df.where("action = 'out'").groupBy('emp_id', "action").agg(expr("max(time) as out_max_time"))

In [41]:
join_df = in_time.join(out_time, in_time["emp_id"] == out_time["emp_id"], 'left')
join_df.show()

+------+------+-------------------+------+------+-------------------+
|emp_id|action|        in_max_time|emp_id|action|       out_max_time|
+------+------+-------------------+------+------+-------------------+
|     1|    in|2019-12-22 09:00:00|     1|   out|2019-12-22 09:15:00|
|     3|    in|2019-12-22 09:45:00|     3|   out|2019-12-22 09:30:00|
|     4|    in|2019-12-22 09:45:00|  NULL|  NULL|               NULL|
|     2|    in|2019-12-22 09:30:00|     2|   out|2019-12-22 09:15:00|
+------+------+-------------------+------+------+-------------------+



In [42]:
join_df.where("in_max_time > out_max_time or out_max_time is null").select(in_time["emp_id"]).show()

+------+
|emp_id|
+------+
|     3|
|     4|
|     2|
+------+

